In [22]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain.prompts.chat import ChatPromptTemplate

In [3]:
loader = WebBaseLoader('https://aws.amazon.com/what-is/ai-agents/')
documents = loader.load()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(documents)

In [7]:
documents

[Document(metadata={'source': 'https://aws.amazon.com/what-is/ai-agents/', 'title': 'What are AI Agents?- Agents in Artificial Intelligence Explained - AWS', 'description': 'What is AI agents how and why businesses use AI agents, and how to use AI agents with AWS.', 'language': 'en-US'}, page_content='What are AI Agents?- Agents in Artificial Intelligence Explained - AWS\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n Skip to main content\n\n\n\n\n\nClick here to return to Amazon Web Services homepage\n\n\n\nAbout AWS\nContact Us\n Support \xa0 \n       \n\n \n English \xa0 \n       \n\n \n My Account \xa0 \n       \n\n \n\n\n\n\n Sign In\n\n\n  Create an AWS Account \n\n\n\n\n\n\n\n   \n\n\n   \n\n\n\n Close \n\n            Profile \n          \n\n            Your profile helps improve your interactions with select AWS experiences. \n          \n\n\n\n Login\n\n\n\n\n\n Close \n\n            Profile \n      

In [10]:
embeddings = OllamaEmbeddings(model='mxbai-embed-large:335m')

In [11]:
vectorstore = Chroma.from_documents(documents=documents, embedding=embeddings)

In [16]:
vectorstore.similarity_search_with_score('what is ai agents')

[(Document(metadata={'description': 'What is AI agents how and why businesses use AI agents, and how to use AI agents with AWS.', 'language': 'en-US', 'source': 'https://aws.amazon.com/what-is/ai-agents/', 'title': 'What are AI Agents?- Agents in Artificial Intelligence Explained - AWS'}, page_content='What are AI Agents?\n\nAn artificial intelligence (AI) agent is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals. Humans set goals, but an AI agent independently chooses the best actions it needs to perform to achieve those goals. For example, consider a contact center AI agent that wants to resolves customer queries. The agent will automatically ask the customer different questions, look up information in internal documents, and respond with a solution. Based on the customer responses, it determines if it can resolve the query itself or pass it on to a human.\nLearn more about what is a

In [23]:
template = """Answer the question based only on the following context:

{context}

Question: {input}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are an helpful assistant'),
        ('user', template)
    ]
)

In [14]:
llm = ChatOllama(model='llama3.2:3b')

In [17]:
retriever = vectorstore.as_retriever()

In [18]:
retriever.invoke('what is ai agents')

[Document(metadata={'description': 'What is AI agents how and why businesses use AI agents, and how to use AI agents with AWS.', 'language': 'en-US', 'source': 'https://aws.amazon.com/what-is/ai-agents/', 'title': 'What are AI Agents?- Agents in Artificial Intelligence Explained - AWS'}, page_content='What are AI Agents?\n\nAn artificial intelligence (AI) agent is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals. Humans set goals, but an AI agent independently chooses the best actions it needs to perform to achieve those goals. For example, consider a contact center AI agent that wants to resolves customer queries. The agent will automatically ask the customer different questions, look up information in internal documents, and respond with a solution. Based on the customer responses, it determines if it can resolve the query itself or pass it on to a human.\nLearn more about what is ar

In [26]:
retriever.batch(['agent' ,'ai'])

[[Document(metadata={'description': 'What is AI agents how and why businesses use AI agents, and how to use AI agents with AWS.', 'language': 'en-US', 'source': 'https://aws.amazon.com/what-is/ai-agents/', 'title': 'What are AI Agents?- Agents in Artificial Intelligence Explained - AWS'}, page_content='How does an AI agent work?'),
  Document(metadata={'description': 'What is AI agents how and why businesses use AI agents, and how to use AI agents with AWS.', 'language': 'en-US', 'source': 'https://aws.amazon.com/what-is/ai-agents/', 'title': 'What are AI Agents?- Agents in Artificial Intelligence Explained - AWS'}, page_content='What are AI Agents?\n\nAn artificial intelligence (AI) agent is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals. Humans set goals, but an AI agent independently chooses the best actions it needs to perform to achieve those goals. For example, consider a conta

In [20]:
from langchain_core.runnables import RunnablePassthrough

In [24]:
chain = (
    {'context' : retriever, 'input' : RunnablePassthrough()}
    | prompt
    | llm 
)

In [25]:
chain.invoke('What is ai agents')

AIMessage(content='According to the provided context, AI Agents are "software programs that can interact with their environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals." They are independent and can choose the best actions needed to achieve those goals.', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-02-15T20:57:46.7155712Z', 'done': True, 'done_reason': 'stop', 'total_duration': 56258223900, 'load_duration': 9008044000, 'prompt_eval_count': 865, 'prompt_eval_duration': 39395000000, 'eval_count': 53, 'eval_duration': 6963000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-8cecf42e-baa7-4571-8e53-fd7801589eec-0', usage_metadata={'input_tokens': 865, 'output_tokens': 53, 'total_tokens': 918})